In [1]:
import pandas as pd
import requests

# URL de base de l'API
base_url = "https://api.star-citizen.wiki/api/v3/vehicles?page="

# Récupérer les données de toutes les pages
all_data = [
    requests.get(f"{base_url}{page}").json().get("data", [])
    for page in range(1, 16)
    if requests.get(f"{base_url}{page}").status_code == 200
]

# Aplatir la liste des listes
all_data = [item for sublist in all_data for item in sublist]

# Créer une DataFrame Pandas avec toutes les données
ships = pd.DataFrame(all_data)

# Supprimer la colonne 'updated_at'
ships.drop(columns=['updated_at', 'version'], inplace=True)

# Afficher la DataFrame
ships


,uuid,name,link
0,6135a874-4cb1-4f49-9f29-5781e5991f2b,100i,https://api.star-citizen.wiki/api/v2/vehicles/...
1,e7eeba8d-b108-48a1-a332-d9edbab2b9c1,125a,https://api.star-citizen.wiki/api/v2/vehicles/...
2,2f231daa-95e5-4cb2-a32e-4db8f29a30d6,135c,https://api.star-citizen.wiki/api/v2/vehicles/...
3,ce937681-caa2-4cfa-ab80-f5bf2a5b9a6c,300i,https://api.star-citizen.wiki/api/v2/vehicles/...
4,dc537d66-2abb-43aa-9d37-4e963fc6e9e2,315p,https://api.star-citizen.wiki/api/v2/vehicles/...
...,...,...,...
215,b416dbbf-53f2-489e-a9dc-456286aa6c70,X1 Force,https://api.star-citizen.wiki/api/v2/vehicles/...
216,58e25524-8bd3-450f-8442-7b58fdfc8a50,X1 Velocity,https://api.star-citizen.wiki/api/v2/vehicles/...
217,None,Zeus Mk II CL,https://api.star-citizen.wiki/api/v2/vehicles/...
218,None,Zeus Mk II ES,https://api.star-citizen.wiki/api/v2/vehicles/...


In [3]:
import pandas as pd
import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def get_vehicle_characteristics(name):
    url = f"https://api.star-citizen.wiki/api/v3/vehicles/{name}?include=components,shops"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()  # Retourne les données sous forme de dictionnaire
    else:
        print(f"Erreur lors de la récupération des données pour {name}: {response.status_code}")
        return None

# Liste pour stocker les données des caractéristiques des vaisseaux
vehicle_names = ships['name'].tolist()

vehicle_data = []

for name in vehicle_names:
    # Gérer les exceptions pour les vaisseaux spécifiques
    if name in ["Carrack Expedition w/C8X", "Carrack w/C8X", "C8 Pisces"]:
        print(f"Skipping {name} due to known issue.")
        continue
    
    characteristics = get_vehicle_characteristics(name)
    print(f"Récupération des caractéristiques pour : {name}")  # Afficher le nom
    if characteristics and "data" in characteristics:
        # Extraire les données de la clé "data"
        data = characteristics["data"]
        
        # Vérifier si les champs nécessaires existent dans les données
        vehicle_info = {
    "Nom du vaisseau": data.get("name"),
    "HP vaisseau": data.get("health"),
    "HP bouclier": data.get("shield_hp"),
    "Capacité de cargo": data.get("cargo_capacity"),
    "Capacité réservoir quantum": data.get("quantum", {}).get("quantum_fuel_capacity"),
    "crew minimum": data.get("crew", {}).get("min"),
    "crew maximum": data.get("crew", {}).get("max"),
    "Type": data.get("type", {}).get("en_EN"),  # Type du vaisseau
    "Classe": data.get("production_status", {}).get("en_EN"),
    "Status de production": data.get("size_class"),  # Statut de production
    "Vitesse SCM": data.get("speed", {}).get("scm"),  # Vitesse SCM
    "Vitesse max": data.get("speed", {}).get("max"),  # Vitesse max
    "Prix de vente": data.get("msrp"),  # Prix
    "lien du pledge": data.get("pledge_url")  # Lien de pledge
}
        vehicle_data.append(vehicle_info)
    else:
        print(f"Pas de caractéristiques valides pour {name}")

# Créer un nouveau DataFrame avec les données collectées
df = pd.DataFrame(vehicle_data)

# Charger les informations d'identification
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('star-citizen-ships-d2b0ff5a2ca4.json', scope)
client = gspread.authorize(creds)

# Ouvrir le Google Sheet
spreadsheet = client.open("Star citizen - ships")  # Remplace par le nom de ton fichier Google Sheet
worksheet = spreadsheet.get_worksheet(0)  # Utilise la première feuille

# Créer un nouveau DataFrame avec les données collectées
df = pd.DataFrame(vehicle_data)
df.fillna("/", inplace=True)
print("Remplissage des NaN")

# Convertir le DataFrame en liste de listes
data_to_insert = [df.columns.values.tolist()] + df.values.tolist()  # Ajouter les noms de colonnes

# Insérer les données dans la feuille Google Sheet
worksheet.clear()  # Efface les données existantes si tu veux partir de zéro
worksheet.update('A1', data_to_insert)  # Insère les données à partir de la cellule A1

Récupération des caractéristiques pour : 100i
Récupération des caractéristiques pour : 125a
Récupération des caractéristiques pour : 135c
Récupération des caractéristiques pour : 300i
Récupération des caractéristiques pour : 315p
Récupération des caractéristiques pour : 325a
Récupération des caractéristiques pour : 350r
Récupération des caractéristiques pour : 400i
Récupération des caractéristiques pour : 600i Explorer
Récupération des caractéristiques pour : 600i Touring
Récupération des caractéristiques pour : 85X
Récupération des caractéristiques pour : 890 Jump
Récupération des caractéristiques pour : A1 Spirit
Récupération des caractéristiques pour : A2 Hercules
Récupération des caractéristiques pour : Anvil Ballista Dunestalker
Récupération des caractéristiques pour : Anvil Ballista Snowblind
Récupération des caractéristiques pour : Apollo Medivac
Récupération des caractéristiques pour : Apollo Triage
Récupération des caractéristiques pour : Ares Inferno
Récupération des caractér

C:\Users\cleme\AppData\Local\Temp\ipykernel_12608\3939541756.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '/' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("/", inplace=True)


Remplissage des NaN


C:\Users\cleme\AppData\Local\Temp\ipykernel_12608\3939541756.py:75: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', data_to_insert)  # Insère les données à partir de la cellule A1


{'spreadsheetId': '1GJH3QmXWMRwRt0_VuUA6e4C-s23h7SlibBCHXAldYGc',
 'updatedRange': "'Feuille 1'!A1:N218",
 'updatedRows': 218,
 'updatedColumns': 14,
 'updatedCells': 3052}